In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install transformers -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [1]:
import pandas as pd

In [3]:
# https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data
data = pd.read_csv("english_preprocessed_exploded.csv")
data.head()

,English_processed,label_task1,label_task2,label_task3
0,Ignore the other one hes an asshole The probl...,YES,REPORTED,OBJECTIFICATION
1,Ignore the other one hes an asshole The probl...,YES,JUDGEMENTAL,OBJECTIFICATION
2,Ignore the other one hes an asshole The probl...,YES,JUDGEMENTAL,SEXUAL-VIOLENCE
3,Ignore the other one hes an asshole The probl...,NO,-,-
4,Ignore the other one hes an asshole The probl...,YES,REPORTED,STEREOTYPING-DOMINANCE


In [ ]:
# data.to_csv("/content/drive/MyDrive/Youtube Tutorials/datasets/toxic_commnets.csv",index=False)

In [4]:
data = data.drop(data[data['label_task3'] == "-"].index)


In [5]:
data['label_task3'].value_counts()

label_task3
STEREOTYPING-DOMINANCE          6770
IDEOLOGICAL-INEQUALITY          5830
OBJECTIFICATION                 5237
MISOGYNY-NON-SEXUAL-VIOLENCE    4130
SEXUAL-VIOLENCE                 3274
UNKNOWN                          227
Name: count, dtype: int64

In [6]:
data = data[['English_processed','label_task3']]
data = data[0:1000]
data.head(10)

,English_processed,label_task3
0,Ignore the other one hes an asshole The probl...,OBJECTIFICATION
1,Ignore the other one hes an asshole The probl...,OBJECTIFICATION
2,Ignore the other one hes an asshole The probl...,SEXUAL-VIOLENCE
4,Ignore the other one hes an asshole The probl...,STEREOTYPING-DOMINANCE
5,Ignore the other one hes an asshole The probl...,SEXUAL-VIOLENCE
6,Ignore the other one hes an asshole The probl...,IDEOLOGICAL-INEQUALITY
7,Ignore the other one hes an asshole The probl...,MISOGYNY-NON-SEXUAL-VIOLENCE
12,If comicsgate is anything like gamergate then...,OBJECTIFICATION
22,A quite unfortunate social delay gamergate tr...,IDEOLOGICAL-INEQUALITY
24,A quite unfortunate social delay gamergate tr...,IDEOLOGICAL-INEQUALITY


In [7]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data["label_task3"]=le.fit_transform(data["label_task3"])

In [8]:
data["label_task3"]

0       2
1       2
2       3
4       4
5       3
       ..
1956    0
1957    4
1958    1
1959    3
1961    4
Name: label_task3, Length: 1000, dtype: int64

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2024-04-21 16:32:43.935972: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 16:32:43.936090: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 16:32:43.975126: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-21 16:32:44.055749: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 16:32:45.430247: W tensorflow/compiler/tf2

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=6)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/bert-base-uncased/68d45e234eb4a928074dfd868cead0219ab85354cc53d20e772753c6bb9169d3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1713953162&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzk1MzE2Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9iZXJ0LWJhc2UtdW5jYXNlZC82OGQ0NWUyMzRlYjRhOTI4MDc0ZGZkODY4Y2VhZDAyMTlhYjg1MzU0Y2M1M2QyMGU3NzI3NTNjNmJiOTE2OWQzP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=W5ZeBfsH66OAqrT2eM1%7Exk0grAr6-bbSQOH5dsoSY7pvIUSkjJ74uBXzZG%7EohM21HzJtGCA%7EMZQh2Za8mzQqLcxJiGeGg3RFYy6XqDRjPtu1Y0QgW8L4cmNp8CHicOsK5xv4MyOSSn0Xk5toBawJiPVSUfiVoiF-2fgH0uLSZgNUOGOyxy15B1%7E05ctaWRRgskT9FV3DmH%7E-QClzgI4M5ynk99091Uv-LTiZCROIUzsFtzcpm1ns%7E47k2sT6515h-bZJdOXAdZG8JdBCwP37wlirUdRaHXvAgFELnzGaKcoHERWgN3s4h%7EqNHHbNw-tut95%7E5iA-HrPY5%7EqIZjUPBw__&Key-Pair-Id=KVTP0A1D

model.safetensors:   7%|7         | 31.5M/440M [00:00<?, ?B/s]

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# device = torch.device('cpu')
# print('CUDA is not available, using CPU for computation.')

In [ ]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
X = list(data["English_processed"])
y = list(data["label_task3"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
len(X_train),len(X_val)

(800, 200)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

{'input_ids': tensor([ 101, 5292, 3270, 3270, 3270, 2308, 2031, 2000, 2293, 2068, 2077, 4824,
         2068, 2204, 2851, 2026, 6203, 4702, 2502, 8549,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# def compute_metrics(p):
#     print(type(p))
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred)
#     precision = precision_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred, average=None)

#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=1.6980471801757813, metrics={'train_runtime': 685.3988, 'train_samples_per_second': 1.167, 'train_steps_per_second': 0.146, 'total_flos': 28778805216000.0, 'train_loss': 1.6980471801757813, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.6152780055999756,
 'eval_runtime': 45.9069,
 'eval_samples_per_second': 4.357,
 'eval_steps_per_second': 0.545,
 'epoch': 1.0}

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
text = "That was good point"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cpu')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.8366,  0.3152,  0.7642,  0.4081,  1.0909, -1.0415]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.2166, 0.1286, 0.2014, 0.1411, 0.2793, 0.0331]],
       grad_fn=<SoftmaxBackward0>)


array([[0.21655971, 0.1285648 , 0.20142774, 0.14107834, 0.27926332,
        0.03310611]], dtype=float32)

In [ ]:
trainer.save_model('CustomModel_task3')

In [ ]:
# trainer.save_model('/content/drive/MyDrive/Youtube Tutorials/toxic')
# model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Youtube Tutorials/toxic")
# model_2.to('cuda')

In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("CustomModel_task3")
model_2.to('cpu')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# text = "That was good point"
text = "Alejandro Saavedra rapist part of  in SCZ finally sentenced to  years"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cpu')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.21819948, 0.16370448, 0.17242908, 0.241199  , 0.17790358,
        0.0265644 ]], dtype=float32)

In [ ]:

index = np.argmax(predictions, axis=1)
print(index[0])

3


In [ ]:
reverse_mapping = dict(zip(le.transform(le.classes_), le.classes_))


In [ ]:
predicted_class_index = ...  # Replace this with the predicted class index from your model
original_label = reverse_mapping[index[0]]
original_label

'SEXUAL-VIOLENCE'

# Pipeline

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

In [ ]:
myModel = BertForSequenceClassification.from_pretrained("CustomModel")
myModel.to('cuda')
def getType(input_text):
  inputs = tokenizer(input_text,padding = True, truncation = True, return_tensors='pt').to('cuda')
  outputs = myModel(**inputs)
  predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
  predictions = predictions.cpu().detach().numpy()
  index = np.argmax(predictions, axis=1)
  return reverse_mapping[index[0]]


In [ ]:
getType(" You got it Lalu  The cover went from liking it a lot to not liking it at all  Seen next to Messiswell yes  I tell you that I identify with her a thousand times more in these photos  But that is not the point  Yes it is microaggression")

'REPORTED'

In [ ]:
sample_data=data.head(10)
sample_data["pred"]=sample_data.apply(lambda x: getType(x["English_processed"]))

KeyError: 'English_processed'

In [ ]:
data["pred"]=data.apply(lambda x: getType(x["English_processed"]))

,English_processed,label_task2
0,Ignore the other one hes an asshole The probl...,2
1,Ignore the other one hes an asshole The probl...,1
2,Ignore the other one hes an asshole The probl...,1
4,Ignore the other one hes an asshole The probl...,2
5,Ignore the other one hes an asshole The probl...,1
